# Imports

In [1]:
pip install -U scikit-learn imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 105.6 MB/s eta 0:00:0000:01:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.6.1 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from sklearn.preprocessing import MinMaxScaler
import torch.optim as optim
from tqdm.notebook import tqdm
from sklearn.ensemble import IsolationForest
from imblearn.over_sampling import SMOTE
from collections import Counter

# Read the data

In [3]:
data_train = pd.read_csv('/kaggle/input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/train_sessions.csv')
data_test = pd.read_csv('/kaggle/input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/test_sessions.csv')

In [4]:
data_train

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
0,1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
2,3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
3,4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
4,5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253556,253557,3474,2013-11-25 10:26:54,3474.0,2013-11-25 10:26:58,141.0,2013-11-25 10:27:03,2428.0,2013-11-25 10:27:04,106.0,...,2013-11-25 10:27:16,2428.0,2013-11-25 10:27:28,2428.0,2013-11-25 10:27:40,2428.0,2013-11-25 10:27:52,148.0,2013-11-25 10:27:53,0
253557,253558,12727,2013-03-12 16:01:15,12727.0,2013-03-12 16:01:16,2215.0,2013-03-12 16:01:16,38.0,2013-03-12 16:01:17,2215.0,...,2013-03-12 16:01:17,25444.0,2013-03-12 16:01:18,2215.0,2013-03-12 16:01:18,23.0,2013-03-12 16:01:18,21.0,2013-03-12 16:01:18,0
253558,253559,2661,2013-09-12 14:05:03,15004.0,2013-09-12 14:05:10,5562.0,2013-09-12 14:05:10,5562.0,2013-09-12 14:06:29,5562.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
253559,253560,812,2013-12-19 15:20:22,676.0,2013-12-19 15:20:22,814.0,2013-12-19 15:20:22,22.0,2013-12-19 15:20:22,39.0,...,2013-12-19 15:20:23,814.0,2013-12-19 15:20:23,570.0,2013-12-19 15:20:23,22.0,2013-12-19 15:20:24,570.0,2013-12-19 15:20:24,0


In [5]:
data_test

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
0,1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,...,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
1,2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,...,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
2,3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,...,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36
3,4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,...,3374.0,2014-11-04 10:03:22,50.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23
4,5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,...,69.0,2014-05-16 15:05:40,70.0,2014-05-16 15:05:40,68.0,2014-05-16 15:05:40,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82792,82793,812,2014-10-02 18:20:09,1039.0,2014-10-02 18:20:09,676.0,2014-10-02 18:20:09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82793,82794,300,2014-05-26 14:16:40,302.0,2014-05-26 14:16:41,302.0,2014-05-26 14:16:44,300.0,2014-05-26 14:16:44,300.0,...,1222.0,2014-05-26 14:17:19,302.0,2014-05-26 14:17:19,1218.0,2014-05-26 14:17:19,1221.0,2014-05-26 14:17:19,1216.0,2014-05-26 14:17:19
82794,82795,29,2014-05-02 11:21:56,33.0,2014-05-02 11:21:56,35.0,2014-05-02 11:21:56,22.0,2014-05-02 11:22:03,37.0,...,6779.0,2014-05-02 11:22:03,30.0,2014-05-02 11:22:03,21.0,2014-05-02 11:22:04,23.0,2014-05-02 11:22:04,6780.0,2014-05-02 11:22:04
82795,82796,5828,2014-05-03 10:05:25,23.0,2014-05-03 10:05:27,21.0,2014-05-03 10:05:27,804.0,2014-05-03 10:05:27,21.0,...,3350.0,2014-05-03 10:05:37,23.0,2014-05-03 10:05:37,894.0,2014-05-03 10:05:38,21.0,2014-05-03 10:05:38,961.0,2014-05-03 10:05:38


In [6]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253561 entries, 0 to 253560
Data columns (total 22 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   session_id  253561 non-null  int64  
 1   site1       253561 non-null  int64  
 2   time1       253561 non-null  object 
 3   site2       250098 non-null  float64
 4   time2       250098 non-null  object 
 5   site3       246919 non-null  float64
 6   time3       246919 non-null  object 
 7   site4       244321 non-null  float64
 8   time4       244321 non-null  object 
 9   site5       241829 non-null  float64
 10  time5       241829 non-null  object 
 11  site6       239495 non-null  float64
 12  time6       239495 non-null  object 
 13  site7       237297 non-null  float64
 14  time7       237297 non-null  object 
 15  site8       235224 non-null  float64
 16  time8       235224 non-null  object 
 17  site9       233084 non-null  float64
 18  time9       233084 non-null  object 
 19  si

В трейне пропуски

In [7]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82797 entries, 0 to 82796
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   session_id  82797 non-null  int64  
 1   site1       82797 non-null  int64  
 2   time1       82797 non-null  object 
 3   site2       81308 non-null  float64
 4   time2       81308 non-null  object 
 5   site3       80075 non-null  float64
 6   time3       80075 non-null  object 
 7   site4       79182 non-null  float64
 8   time4       79182 non-null  object 
 9   site5       78341 non-null  float64
 10  time5       78341 non-null  object 
 11  site6       77566 non-null  float64
 12  time6       77566 non-null  object 
 13  site7       76840 non-null  float64
 14  time7       76840 non-null  object 
 15  site8       76151 non-null  float64
 16  time8       76151 non-null  object 
 17  site9       75484 non-null  float64
 18  time9       75484 non-null  object 
 19  site10      74806 non-nul

# Fill missing

In [8]:
data_train.fillna(0, inplace=True)
data_test.fillna(0, inplace=True)

Заполняем всё нулями

# Feature engineering

In [9]:
def add_time_diffs_at_seconds(df):
    time_cols = sorted([col for col in df.columns if col.startswith('time')])
    data_test[time_cols] = data_test[time_cols].apply(pd.to_datetime, errors='coerce')
    data_train[time_cols] = data_train[time_cols].apply(pd.to_datetime, errors='coerce')

    for i in range(len(time_cols)-1):
        col1 = time_cols[i]
        col2 = time_cols[i+1] 
        diff_col = f'diff{col1}_{col2}'

        df[diff_col] = (df[col2]-df[col1]).dt.total_seconds()

    return df

Находим разницу во времени входа на сайт в секундах

In [10]:
data_test = add_time_diffs_at_seconds(data_test)
data_train = add_time_diffs_at_seconds(data_train)

In [11]:
def time_features_for_alice(df):
    time_cols = sorted([col for col in df.columns if col.startswith('time')])
    data_test[time_cols] = data_test[time_cols].apply(pd.to_datetime, errors='coerce')
    data_train[time_cols] = data_train[time_cols].apply(pd.to_datetime, errors='coerce')


    for i in range(len(time_cols)):
        
        col = time_cols[i]
        df[f'year_{col}'] = df[col].dt.year
        df[f'month_{col}'] = df[col].dt.month
        df[f'day_{col}'] = df[col].dt.day
        df[f'hour_{col}'] = df[col].dt.hour
        df[f'minute_{col}'] = df[col].dt.minute
        df[f'second_{col}'] = df[col].dt.second
        df[f'weekday_{col}'] = df[col].dt.dayofweek
        df = df.drop(col, axis=1)

    return df

Распарсим дату

In [12]:
data_test = time_features_for_alice(data_train)
data_train = time_features_for_alice(data_test)

In [13]:
def cap_large_values_in_time_cols(df, threshold=864000):

    time_cols = [col for col in df.columns if col.startswith('diff_')]
    
    for col in time_cols:
        mask = df[col].abs() > threshold
        df.loc[mask, col] = threshold * df.loc[mask, col].apply(lambda x: 1 if x > 0 else -1)
    
    return df

In [14]:
data_test = cap_large_values_in_time_cols(data_train)
data_train = cap_large_values_in_time_cols(data_test)

In [15]:
features = ['year_time1', 'month_time1', 'day_time1', 'weekday_time1', 'hour_time1',
       'minute_time1', 'second_time1', 'year_time2', 'month_time2',
       'day_time2', 'weekday_time2', 'hour_time2', 'minute_time2',
       'second_time2', 'year_time3', 'month_time3', 'day_time3',
       'weekday_time3', 'hour_time3', 'minute_time3', 'second_time3',
       'year_time4', 'month_time4', 'day_time4', 'weekday_time4', 'hour_time4',
       'minute_time4', 'second_time4', 'year_time5', 'month_time5',
       'day_time5', 'weekday_time5', 'hour_time5', 'minute_time5',
       'second_time5', 'year_time6', 'month_time6', 'day_time6',
       'weekday_time6', 'hour_time6', 'minute_time6', 'second_time6',
       'year_time7', 'month_time7', 'day_time7', 'weekday_time7', 'hour_time7',
       'minute_time7', 'second_time7', 'year_time8', 'month_time8',
       'day_time8', 'weekday_time8', 'hour_time8', 'minute_time8',
       'second_time8', 'year_time9', 'month_time9', 'day_time9',
       'weekday_time9', 'hour_time9', 'minute_time9', 'second_time9',
       'year_time10', 'month_time10', 'day_time10', 'weekday_time10',
       'hour_time10', 'minute_time10', 'second_time10']


data_train = pd.get_dummies(data_train, columns=features, dtype=int)

# Data exploration

In [16]:
data_anomaly = data_train.loc[(data_train['target']==1)]

In [17]:
data_not_anomaly = data_train.loc[(data_train['target']==0)]

In [18]:
data_not_anomaly = data_not_anomaly.drop('session_id', axis=1)

In [19]:
X = data_not_anomaly.drop('target', axis=1)
y = data_not_anomaly['target']

In [20]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Dataset to tensors

In [21]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [22]:
train_dataset = TensorDataset(X,y)

In [23]:
train_loader = DataLoader(train_dataset, batch_size = 256, shuffle = True)

In [24]:
X.shape

torch.Size([251264, 1867])

# Encoder

In [29]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.LazyLinear(1024),
            nn.ReLU(),
            nn.LazyLinear(512),
            nn.ReLU(),
            nn.LazyLinear(256),
            nn.ReLU(),
            nn.LazyLinear(64),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.LazyLinear(256),
            nn.ReLU(),
            nn.LazyLinear(512),
            nn.ReLU(),
            nn.LazyLinear(1024),
            nn.ReLU(),
            nn.LazyLinear(1867)
        )

    def forward(self,x):
        out = self.encoder(x)
        out = self.decoder(out)
        return out

In [30]:
model = Encoder()

In [31]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [32]:
epochs = 15
for epoch in tqdm(range(epochs)):
    model.train()
    running_loss = 0.0
    for data, labels in train_loader:
        optimizer.zero_grad()

        outputs = model(data)
        loss = criterion(outputs, data)
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'epoch {epoch + 1} / {epochs}, Loss: {running_loss / len(train_loader)}')

  0%|          | 0/15 [00:00<?, ?it/s]

epoch 1 / 15, Loss: 0.01476637507047193
epoch 2 / 15, Loss: 0.010679715939905578
epoch 3 / 15, Loss: 0.009218006551762675
epoch 4 / 15, Loss: 0.007154038085843116
epoch 5 / 15, Loss: 0.006756851277475354
epoch 6 / 15, Loss: 0.006531213562890068
epoch 7 / 15, Loss: 0.006548108480863136
epoch 8 / 15, Loss: 0.00619571211713683
epoch 9 / 15, Loss: 0.006029628111000216
epoch 10 / 15, Loss: 0.006327042696132588
epoch 11 / 15, Loss: 0.0059288280239317365
epoch 12 / 15, Loss: 0.005795305889278134
epoch 13 / 15, Loss: 0.006716818946590097
epoch 14 / 15, Loss: 0.006067919893410292
epoch 15 / 15, Loss: 0.006140404048445966


In [33]:
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, 'model.pth')

In [34]:
model.eval()

Encoder(
  (encoder): Sequential(
    (0): Linear(in_features=1867, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=64, bias=True)
    (7): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=64, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): ReLU()
    (6): Linear(in_features=1024, out_features=1867, bias=True)
  )
)

# Analyizing losses

In [35]:
def analos(model, dataloader):
    criterion = nn.MSELoss(reduction='none')
    losses = []
    with torch.no_grad():

        for X_batch, _ in dataloader:
            output = model(X_batch)
            loss = criterion(X_batch, output)
            losses.extend(loss.mean(dim=1).tolist())

    return losses

In [36]:
losses = analos(model, train_loader)

In [37]:
max(losses)

0.01619568094611168

In [38]:
type(losses[90])

float

# Create new dataset

In [39]:
y = data_train['target']
X = data_train.drop(['target', 'session_id'], axis=1)

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.8)

In [42]:
def inference(X, y, model):
    X = scaler.transform(X)
    
    y = torch.tensor(y, dtype=torch.float32)
    X = torch.tensor(X, dtype=torch.float32)

    train_dataset = TensorDataset(X, y)
    train_loader = DataLoader(train_dataset, batch_size=256, shuffle = True)

    criterion = nn.MSELoss(reduction='none')

    model.eval()
    pred=[]
    loss_data=[]

    with torch.no_grad():

        for data, _ in train_loader:
            pred.append(model.encoder(data))
            output = model(data)
            loss = criterion(data, output)
            loss_data.extend(loss.mean(dim=1).tolist())

    pred = torch.cat(pred, dim=0).numpy()
    loss_frame = pd.DataFrame(data={'loss': loss_data})

    data_orig = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
    data_pred = pd.DataFrame(pred, columns=[f'reconstructed_{i}' for i in range(pred.shape[1])])
    data = pd.concat([data_orig, data_pred], axis=1)
    data = pd.concat([data, loss_frame], axis=1)

    return data

In [43]:
train_X = inference(X_train, y_train, model)

In [44]:
train_X

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,reconstructed_55,reconstructed_56,reconstructed_57,reconstructed_58,reconstructed_59,reconstructed_60,reconstructed_61,reconstructed_62,reconstructed_63,loss
0,0.184736,0.068438,0.049280,0.038150,0.091829,0.038149,0.038125,0.016274,0.000913,0.000793,...,1.059280,0.000000,1.859702,0.0,0.028186,1.471236,0.0,1.344402,0.0,0.006121
1,0.021106,0.021130,0.001923,0.001923,0.021130,0.021130,0.021130,0.018846,0.001923,0.021129,...,0.249214,0.558474,0.892837,0.0,0.000000,0.804739,0.0,1.063437,0.0,0.004643
2,0.003990,0.000072,0.119546,0.119498,0.119522,0.004014,0.000913,0.004014,0.939641,0.939641,...,2.207727,0.264195,3.197790,0.0,0.000000,1.897879,0.0,0.970475,0.0,0.006761
3,0.008053,0.000697,0.000721,0.000721,0.000889,0.008077,0.000553,0.000553,0.000553,0.000505,...,0.477425,0.237988,0.316234,0.0,0.000000,0.423263,0.0,0.597685,0.0,0.001421
4,0.983149,0.983558,0.983197,0.983581,0.983197,0.983558,0.983173,0.983534,0.983534,0.983149,...,0.709589,0.376021,0.514445,0.0,0.000000,0.208352,0.0,0.681272,0.0,0.002239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202843,0.016226,0.000505,0.005817,0.000721,0.004351,0.005817,0.000529,0.000553,0.000769,0.012404,...,1.330422,0.681266,1.860930,0.0,0.221023,1.422324,0.0,1.106772,0.0,0.005889
202844,0.000673,0.624471,0.009616,0.009616,0.010818,0.021538,0.224591,0.272445,0.066104,0.021538,...,1.411300,0.825056,1.740891,0.0,0.718072,0.754348,0.0,1.647013,0.0,0.006349
202845,0.001563,0.001611,0.001659,0.001683,0.001635,0.001707,0.001659,0.001611,0.001683,0.001514,...,1.436762,0.000000,1.667992,0.0,0.109421,1.349706,0.0,0.637127,0.0,0.008223
202846,0.000913,0.019519,0.018799,0.000529,0.018606,0.018630,0.014808,0.018894,0.000505,0.018798,...,0.978363,0.837290,1.945233,0.0,0.000000,1.114578,0.0,1.634580,0.0,0.005325


# Data sampling

In [45]:
smote = SMOTE(random_state = 42, sampling_strategy={1: 5_000})
X_smote, y_smote = smote.fit_resample(train_X, y_train)

In [46]:
X_smote

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,reconstructed_55,reconstructed_56,reconstructed_57,reconstructed_58,reconstructed_59,reconstructed_60,reconstructed_61,reconstructed_62,reconstructed_63,loss
0,0.184736,0.068438,0.049280,0.038150,0.091829,0.038149,0.038125,0.016274,0.000913,0.000793,...,1.059280,0.000000,1.859702,0.0,0.028186,1.471236,0.0,1.344402,0.0,0.006121
1,0.021106,0.021130,0.001923,0.001923,0.021130,0.021130,0.021130,0.018846,0.001923,0.021129,...,0.249214,0.558474,0.892837,0.0,0.000000,0.804739,0.0,1.063437,0.0,0.004643
2,0.003990,0.000072,0.119546,0.119498,0.119522,0.004014,0.000913,0.004014,0.939641,0.939641,...,2.207727,0.264195,3.197790,0.0,0.000000,1.897879,0.0,0.970475,0.0,0.006761
3,0.008053,0.000697,0.000721,0.000721,0.000889,0.008077,0.000553,0.000553,0.000553,0.000505,...,0.477425,0.237988,0.316234,0.0,0.000000,0.423263,0.0,0.597685,0.0,0.001421
4,0.983149,0.983558,0.983197,0.983581,0.983197,0.983558,0.983173,0.983534,0.983534,0.983149,...,0.709589,0.376021,0.514445,0.0,0.000000,0.208352,0.0,0.681272,0.0,0.002239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206004,0.022620,0.027055,0.088019,0.136645,0.088021,0.088017,0.027056,0.136638,0.027077,0.088039,...,1.060208,0.000000,1.049997,0.0,0.131973,1.291487,0.0,1.002022,0.0,0.010680
206005,0.005062,0.021817,0.027010,0.001831,0.001852,0.021320,0.001872,0.001923,0.001963,0.001851,...,0.140564,0.571858,1.049393,0.0,0.000000,0.917866,0.0,1.191259,0.0,0.005202
206006,0.007643,0.003740,0.009770,0.517790,0.016010,0.000838,0.136920,0.000534,0.000754,0.013841,...,0.787115,0.359889,1.877786,0.0,0.464217,0.634405,0.0,0.598284,0.0,0.005450
206007,0.013639,0.195829,0.061388,0.175047,0.195661,0.564834,0.060737,0.026839,0.068313,0.368682,...,1.094896,0.449911,1.721116,0.0,0.000000,1.954827,0.0,1.368033,0.0,0.005418


# Catboost

In [47]:
class_counts = Counter(y_smote)
total = sum(class_counts.values())
class_weights = {cls: total / count for cls, count in class_counts.items()}
class_weights_list = [class_weights[cls] for cls in sorted(class_weights.keys())]

In [48]:
cat = CatBoostClassifier(
    iterations=5_000, 
    learning_rate=0.01, 
    depth=7, verbose=100, 
    random_state=42, 
    class_weights=class_weights_list)

In [49]:
cat.fit(X_smote, y_smote)

0:	learn: 0.6830427	total: 359ms	remaining: 29m 56s
100:	learn: 0.2760021	total: 26s	remaining: 21m 2s
200:	learn: 0.1921233	total: 53.2s	remaining: 21m 11s
300:	learn: 0.1533686	total: 1m 19s	remaining: 20m 46s
400:	learn: 0.1311055	total: 1m 46s	remaining: 20m 18s
500:	learn: 0.1166100	total: 2m 12s	remaining: 19m 45s
600:	learn: 0.1060722	total: 2m 38s	remaining: 19m 16s
700:	learn: 0.0981181	total: 3m 4s	remaining: 18m 48s
800:	learn: 0.0925838	total: 3m 29s	remaining: 18m 17s
900:	learn: 0.0872680	total: 3m 55s	remaining: 17m 50s
1000:	learn: 0.0829714	total: 4m 20s	remaining: 17m 19s
1100:	learn: 0.0793028	total: 4m 46s	remaining: 16m 52s
1200:	learn: 0.0752619	total: 5m 12s	remaining: 16m 28s
1300:	learn: 0.0712155	total: 5m 38s	remaining: 16m 2s
1400:	learn: 0.0671370	total: 6m 5s	remaining: 15m 38s
1500:	learn: 0.0635837	total: 6m 31s	remaining: 15m 12s
1600:	learn: 0.0604941	total: 6m 58s	remaining: 14m 47s
1700:	learn: 0.0576741	total: 7m 24s	remaining: 14m 22s
1800:	learn: 

In [50]:
test_X = inference(X_test, y_test.values, model)

In [51]:
test_X

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,reconstructed_55,reconstructed_56,reconstructed_57,reconstructed_58,reconstructed_59,reconstructed_60,reconstructed_61,reconstructed_62,reconstructed_63,loss
0,0.018822,0.019712,0.019712,0.019712,0.019712,0.018846,0.018846,0.000000,0.000000,0.000000,...,0.684103,0.594184,1.329555,0.0,0.000000,0.928547,0.0,0.696979,0.0,0.009708
1,0.001370,0.001418,0.006491,0.028150,0.000072,0.000192,0.000072,0.000192,0.000072,0.000337,...,0.449521,0.511426,2.480895,0.0,0.000000,1.131762,0.0,0.125530,0.0,0.005163
2,0.019543,0.014808,0.019568,0.014808,0.019568,0.014808,0.019567,0.019567,0.014831,0.019519,...,0.417019,0.396479,0.479783,0.0,0.000000,0.260548,0.0,0.954126,0.0,0.005823
3,0.025793,0.025817,0.025818,0.003221,0.003221,0.001587,0.001514,0.007356,0.007332,0.007356,...,0.906646,0.000000,1.862115,0.0,0.121723,1.143952,0.0,1.296896,0.0,0.004761
4,0.000481,0.000505,0.000505,0.004207,0.000505,0.004255,0.004207,0.004255,0.004279,0.004231,...,1.027317,1.021231,2.142329,0.0,0.000000,1.400759,0.0,1.873962,0.0,0.004658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50708,0.020481,0.064014,0.020529,0.062670,0.062718,0.063462,0.062572,0.020504,0.020504,0.062811,...,0.588844,0.610203,0.908841,0.0,0.174163,1.426402,0.0,1.333161,0.0,0.005405
50709,0.046418,0.046418,0.046492,0.046347,0.046660,0.046587,0.046394,0.046369,0.046609,0.046513,...,1.531924,0.000000,1.513447,0.0,0.659476,1.152758,0.0,0.472778,0.0,0.006041
50710,0.001803,0.233894,0.075507,0.001851,0.001827,0.001851,0.001971,0.001851,0.001923,0.001971,...,0.936256,0.000000,0.498105,0.0,1.422307,1.540824,0.0,0.826606,0.0,0.005313
50711,0.000673,0.000721,0.000625,0.003365,0.000697,0.000625,0.003365,0.000505,0.096969,0.004351,...,0.400812,0.481540,1.038817,0.0,0.000000,0.902009,0.0,1.402511,0.0,0.005787


# Results

In [52]:
predictions = cat.predict(test_X)

In [53]:
roc_auc_score(y_test, predictions)

0.9433544382166237

In [54]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99     50255
           1       0.39      0.90      0.54       458

    accuracy                           0.99     50713
   macro avg       0.69      0.94      0.77     50713
weighted avg       0.99      0.99      0.99     50713



In [55]:
cat.save_model('cat_model.cbm')